In [ ]:
#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas as pd #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`
import csv

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python

import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

# nltk dependencies 
import nltk
import ssl

#pubmed specific stuff
from pubmed_lookup import PubMedLookup
from pubmed_lookup import Publication
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## Scraping using PubMed_Lookup

In [9]:
pmids_list = []
with open('week-1/pmids_na_omit.txt') as csvfile:
    pmids = csv.reader(csvfile, delimiter=',')
    for row in pmids:
        for i in row: 
            pmids_list.append(i)
pmids_list[:5]

['30784590', '30692680', '30659290', '30609404', '30445657']

In [ ]:
all_links = []
for pmid in pmids_list: 
    all_links.append("http://www.ncbi.nlm.nih.gov/pubmed/%s" % pmid)
all_links[:5]

In [ ]:
# THIS IS A SCRAPE 
# abs_corpus = []
# for link in all_links: 
#     try: 
#         email = 'deblina@uchicago.edu'
#         lookup = PubMedLookup(link, email)
#         publication = Publication(lookup) 
#         abs_corpus.append(publication)
#     except TypeError: 
#         pass 
#     except RuntimeError: 
#         pass
#     except TimeoutError: 
#         pass

In [ ]:
# CLEANING INTO DICTIONARY 
# dic = {}
# for pub in abs_corpus: 
#     dic[pub.title] = [pub.abstract]

In [ ]:
# SAVING TO CSV 
# with open('more_corp.csv', 'w') as f:  # Just use 'w' mode in 3.x
#     w = csv.DictWriter(f, dic.keys())
#     w.writeheader()
#     w.writerow(dic)

## Scraping w. Entrez API
Want to have: 
- Major Topic 
- Abstract 
- Title 
- MESH Terms 

for every PMID. 

In [153]:
class Publication: 
    def _init_(self, pmid): 
        self.pmid = pmid
        self.title = None 
        self.abstract = None 
        self.major_topics = None
        self.mesh_terms = None
    def fill_attributes(pmid): 
        handle = en.efetch(db="pubmed",id=pmid,rettype="null",retmode="xml")
        record = Entrez.read(handle)
        self.title = record['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle'] 
        self.abstract = record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0]
        
        self.major_topics = record['PubmedArticle'][0]['MedlineCitation']['KeywordList'][0]
        self.mesh_terms = record['PubmedArticle'][0]['MedlineCitation']['MeshHeadingList']

In [ ]:
from Bio import Entrez as en
en.email = "deblina@uchicago.edu"
en.api_key = "0de728102b2b58a83f3986d583c23f766008" 
en.sleep_between_tries = 10 

In [133]:
handle = en.efetch(db="pubmed",id=pmids_list[0],rettype="null",retmode="xml")
record = Entrez.read(handle)

In [134]:
record['PubmedArticle'][0]['MedlineCitation']['Article']['Abstract']['AbstractText'][0] #abstract 

'Large cohorts of human induced pluripotent stem cells (iPSCs) from healthy donors are a potentially powerful tool for investigating the relationship between genetic variants and cellular behavior. Here, we integrate high content imaging of cell shape, proliferation, and other phenotypes with gene expression and DNA sequence datasets from over 100 human iPSC lines. By applying a dimensionality reduction approach, Probabilistic Estimation of Expression Residuals (PEER), we extracted factors that captured the effects of intrinsic (genetic concordance between different cell lines from the same donor) and extrinsic (cell responses to different fibronectin concentrations) conditions. We identify genes that correlate in expression with intrinsic and extrinsic PEER factors and associate outlier cell behavior with genes containing rare deleterious non-synonymous SNVs. Our study, thus, establishes a strategy for examining the genetic basis of inter-individual variability in cell behavior.'

In [135]:
record['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle'] #title

'Identifying Extrinsic versus Intrinsic Drivers of Variation in Cell Behavior in Human iPSC Lines from Healthy Donors.'

In [136]:
record['PubmedArticle'][0]['MedlineCitation']['KeywordList'][0]

ListElement([StringElement('SNV', attributes={'MajorTopicYN': 'Y'}), StringElement('cell adhesion', attributes={'MajorTopicYN': 'Y'}), StringElement('dimensionality reduction', attributes={'MajorTopicYN': 'Y'}), StringElement('fibronectin', attributes={'MajorTopicYN': 'Y'}), StringElement('genetic variation', attributes={'MajorTopicYN': 'Y'}), StringElement('high content imaging', attributes={'MajorTopicYN': 'Y'}), StringElement('iPSC', attributes={'MajorTopicYN': 'Y'}), StringElement('stem cell niche', attributes={'MajorTopicYN': 'Y'}), StringElement('stem cells', attributes={'MajorTopicYN': 'Y'})], attributes={'Owner': 'NOTNLM'})

In [138]:
handle = en.efetch(db='pubmed', id=pmids_list[1], retmode='xml')
xml_data = en.read(handle)
xml_data['PubmedArticle'][0]['MedlineCitation']['MeshHeadingList']

[{'QualifierName': [StringElement('genetics', attributes={'UI': 'Q000235', 'MajorTopicYN': 'Y'})], 'DescriptorName': StringElement('Disease', attributes={'UI': 'D004194', 'MajorTopicYN': 'N'})}, {'QualifierName': [StringElement('genetics', attributes={'UI': 'Q000235', 'MajorTopicYN': 'Y'})], 'DescriptorName': StringElement('Genome', attributes={'UI': 'D016678', 'MajorTopicYN': 'N'})}, {'QualifierName': [StringElement('methods', attributes={'UI': 'Q000379', 'MajorTopicYN': 'N'})], 'DescriptorName': StringElement('Genome-Wide Association Study', attributes={'UI': 'D055106', 'MajorTopicYN': 'N'})}, {'QualifierName': [StringElement('methods', attributes={'UI': 'Q000379', 'MajorTopicYN': 'N'})], 'DescriptorName': StringElement('Genomics', attributes={'UI': 'D023281', 'MajorTopicYN': 'N'})}, {'QualifierName': [], 'DescriptorName': StringElement('Humans', attributes={'UI': 'D006801', 'MajorTopicYN': 'N'})}, {'QualifierName': [StringElement('methods', attributes={'UI': 'Q000379', 'MajorTopicYN

**TO DO:**
1. Put into a function or two 
2. Run on whole list 
3. Cluster 